# Polyolefin InfraRed Classification - Nested Cross-Validation Analysis for identification and evaluation of a consitently accurate Polyolefin-IR-Classifier

In connection with: add DOI INFO/LINK HERE

This code was predominantly produced by Bradley P. Sutliff, with assistance from Tyler B. Martin, and Debra Audus

This notebook is provided in an effort to further open research initiatives and to further the circular economy.

Please direct any questions to Bradley.Sutliff@nist.gov

This notebook relies on running 3 different python scripts that can take over a week to run to train, test, and evaluate the models. It is highly recommended to

Your results may vary depending on the final options you chose to use to get this far. I also did not set (random) seeds, which will purposefully introduce some level of variation between users/computers/runs.

At this point, your file directory should look something similar to:

```
Main  
  ├ *.ipynb  
  ├ Data  
  |  ├ SampleInformation.csv  
  |  └ NIR  
  |    ├ N1476LDPE_1.csv  
  |    ├ ...  
  |    └ H0009PP_7.csv  
  ├ Scripts  
  |  ├ *.py  
  |  ├ *.sh  
  |  └ *.ps1
  ├ NetCDFs
  |   └ *.nc
  ├ ClassifierScores
  |   ├ AllColors-AllStates_None1_None2_None3_None4_None5_None6.csv
  |   ├ ...
  |   └ AllColors-AllStates_RNV_Detrending_SG2_L2_StandardScaler_UMAP.csv
  └ Figures
     └ Examples
       └ *.svg

```

## Set some plotting/saving variables for later

In [1]:
# Change to True if you want to export and save figures
save_figs = True
# Set image type for saving
img_type = "svg"
# set image style to late refer to our custom matplotlib style sheet
img_style = 'paper'

## Selecting the the most consistent preprocessing pipeline

To limit the computational resources necessarry to evaluate all possible models with hyperparameter tuning, we will select only a few pipelines from the results of notebooks 3 and 4. 

#### $\color{red}{\text{Please run}}$ `5a-pirc_classification-PreProcessingSelection.py` $\color{red}{\text{before continuing!}}$

In [2]:
# os imports
import datetime
import os
import pathlib
import re

### Grab data for identifying best preprocessing steps using Random Forest

In [3]:
from os import listdir
from os.path import isfile, join

file_path = r'HP_Param_Scores/Optuna/Example/'
filelist = [f for f in listdir(file_path) if isfile(join(file_path, f))]
pp_ol = [f for f in filelist if 'OuterLoop' in f]
pp_il = [f for f in filelist if 'OStudy' in f]

print(f'Files: {len(pp_ol)}')
print(f'{pp_ol[0]}') # just take a look at a random file to make sure we are sorting properly

Files: 10
20240625172254_OuterLoop_8.csv


In [4]:
import pandas as pd

pp_df = pd.DataFrame()
for file in pp_ol:
    data = pd.read_csv(file_path+file)
    cols_to_fix = [col for col in data.columns
                   if ("Inner" in col) or ("Outer" in col)]
    fixed_cols = ['_'.join(col.split('_')[:-1]) for col in cols_to_fix]
    fix_dict = dict(zip(cols_to_fix, fixed_cols))
    data.rename(columns=fix_dict, inplace=True)

    data['Classifier'] = 'Random Forest'
    pp_df = pd.concat([pp_df, data])
pp_df.sort_values('Outer_acc', ascending=False)

,Unnamed: 0,pp1,pp2,pp3,pp4,pp5,dr,criterion,max_depth,n_estimators,max_features,min_samples_leaf,bootstrap,Inner_acc,Outer_acc,Outer_acc_ball,Outer_f1,Outer_prec,Outer_rec,Classifier
0,9,RNV,None2,None3,None4,StandardScaler,None6,gini,8,678,sqrt,2,False,0.925926,0.967742,0.833333,0.953917,0.943548,0.967742,Random Forest
0,4,RNV,None2,None3,None4,None5,None6,gini,11,586,sqrt,5,False,0.918519,0.967742,0.984848,0.967675,0.970430,0.967742,Random Forest
0,6,RNV,None2,None3,None4,MinMaxScaler,None6,gini,11,796,sqrt,4,False,0.944444,0.967742,0.833333,0.952314,0.938172,0.967742,Random Forest
0,3,RNV,None2,None3,None4,StandardScaler,None6,entropy,19,453,sqrt,3,False,0.925926,0.967742,0.984848,0.967675,0.970430,0.967742,Random Forest
0,1,RNV,None2,None3,None4,StandardScaler,None6,gini,43,184,sqrt,2,False,0.937037,0.967742,0.833333,0.952314,0.938172,0.967742,Random Forest
0,8,RNV,None2,None3,None4,None5,None6,entropy,26,127,sqrt,5,False,0.940741,0.935484,0.818182,0.926651,0.929032,0.935484,Random Forest
0,7,RNV,None2,None3,None4,MinMaxScaler,None6,gini,21,522,log2,1,False,0.940741,0.935484,0.818182,0.922683,0.917742,0.935484,Random Forest
0,5,RNV,None2,None3,None4,MinMaxScaler,None6,gini,28,525,sqrt,5,False,0.951852,0.903226,0.651515,0.875192,0.856700,0.903226,Random Forest
0,2,RNV,None2,None3,None4,None5,None6,entropy,6,986,sqrt,3,False,0.951852,0.903226,0.625000,0.873656,0.856700,0.903226,Random Forest
0,0,RNV,None2,None3,None4,None5,None6,gini,18,341,sqrt,1,False,0.948148,0.870968,0.872475,0.874454,0.892473,0.870968,Random Forest


For my full dataset, this reveals that RNV + MinMaxScaler was the best pre-processing step for 6 of the 10 loops we performed (see Table 3 of the corresponding manuscript) . This is impressive given the wide variety of potential preprocessing combinations and the flexibility of the RandomForest classifier. Please note that according to the [scikit learn docs](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html), tree based models are mostly unaffected by feature scaling. This means that our choice of a random forest classifier will undervalue feature scaling compared to other models.

Your results may vary based on any down-selecting you may have done, and some general variation from random seeds or package versions.

## Selecting the best classification algorithm

Based on my results, I will move forward with using RNV + MinMaxScaler, since RNV consistently performs very well, and scaling can drastically help many of the classification algorithms.

### $\color{red}{\text{Please run}}$ `5b-pirc_classification-AlgorithmSelection.py` $\color{red}{\text{before continuing!}}$

### Make our file lists, for inner and outer loop results

In [5]:

file_path = r'HP_Param_Scores/Optuna/Example/IndvModel/'
filelist = [f for f in listdir(file_path) if isfile(join(file_path, f))]
just_ol = [f for f in filelist if 'OuterLoop' in f]
just_il = [f for f in filelist if 'OStudy' in f]

print(f'Files: {len(just_ol)}')
print(f'{just_ol[0]}')


Files: 90
20240626105709_OuterLoop_LDA_7.csv


## Bring in the outerloop data

In [6]:
import pandas as pd

df = pd.DataFrame()
for file in just_ol:
    data = pd.read_csv(file_path+file)
    data['Classifier'] = file.split('_')[2]
    df = pd.concat([df, data])
df.head()

,Unnamed: 0,n_components,shrinkage,solver,tol,Inner_acc,Outer_acc,Outer_acc_bal,Outer_f1,Outer_prec,...,bootstrap,kernel,gamma,weights,algorithm,leaf_size,n_neighbors,p,estimator,var_smoothing
0,7,3.0,NaN,svd,0.012455,0.911111,0.935484,0.969697,0.935484,0.935484,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1,NaN,NaN,NaN,0.072171,0.740741,0.580645,0.299242,0.527543,0.590323,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1,NaN,NaN,NaN,0.012912,0.955556,0.838710,0.843434,0.827445,0.856824,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,NaN,NaN,lbfgs,0.005318,0.925926,0.935484,0.666667,0.906231,0.881720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3,NaN,NaN,lbfgs,0.001075,0.907407,0.935484,0.791667,0.920056,0.913978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reorganize our DataFrame so it is a little easier to see the values we are interested in

In [7]:
cols_to_move = ['Classifier','Outer_acc', 'Inner_acc', 'Outer_acc_bal',
                'Outer_f1', 'Outer_prec','Outer_rec']
df = df[ cols_to_move + [ col for col in df.columns if col not in cols_to_move] ]

df.sort_values(by=['Inner_acc', 'Outer_acc'], ascending=False).head(10)

,Classifier,Outer_acc,Inner_acc,Outer_acc_bal,Outer_f1,Outer_prec,Outer_rec,Unnamed: 0,n_components,shrinkage,...,bootstrap,kernel,gamma,weights,algorithm,leaf_size,n_neighbors,p,estimator,var_smoothing
0,AdaBoost,0.935484,0.966667,0.929293,0.940092,0.951613,0.935484,6,NaN,NaN,...,NaN,NaN,NaN,NaN,SAMME,NaN,NaN,NaN,"DecisionTreeClassifier(max_depth=5, max_featur...",NaN
0,AdaBoost,0.935484,0.962963,0.666667,0.904628,0.876344,0.935484,2,NaN,NaN,...,NaN,NaN,NaN,NaN,SAMME,NaN,NaN,NaN,"DecisionTreeClassifier(max_depth=3, max_featur...",NaN
0,LinearSVC,1.000000,0.959259,1.000000,1.000000,1.000000,1.000000,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,RandomForest,0.967742,0.955556,0.984848,0.969278,0.975806,0.967742,6,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,LinearSVC,0.838710,0.955556,0.843434,0.827445,0.856824,0.838710,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AdaBoost,0.838710,0.955556,0.705808,0.827189,0.821340,0.838710,1,NaN,NaN,...,NaN,NaN,NaN,NaN,SAMME,NaN,NaN,NaN,"DecisionTreeClassifier(max_depth=4, max_featur...",NaN
0,AdaBoost,0.967742,0.951852,0.833333,0.953917,0.943548,0.967742,8,NaN,NaN,...,NaN,NaN,NaN,NaN,SAMME,NaN,NaN,NaN,"DecisionTreeClassifier(max_depth=4, max_featur...",NaN
0,RandomForest,0.935484,0.951852,0.902778,0.936032,0.954301,0.935484,0,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AdaBoost,0.967742,0.951852,0.984848,0.968766,0.974194,0.967742,7,NaN,NaN,...,NaN,NaN,NaN,NaN,SAMME,NaN,NaN,NaN,"DecisionTreeClassifier(max_depth=3, max_featur...",NaN
0,RandomForest,0.870968,0.951852,0.912879,0.870968,0.870968,0.870968,1,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Use the `.agg` and `.sort_values` methods to view the top performing algorithms, based on average outer-loop score

The outer-loop score evaluates how well and how consistently the algorithm performs on various subsets of our data. For each iteration of the loop the algorithm is retrained (hyperparameters are adjusted) for a slightly different set of data. This means each loop has a slightly different model, but since they are all optimized using the same tools and limits, they should be similar.

In [8]:
df[cols_to_move].groupby('Classifier').agg(['mean', 'max', 'std']).sort_values(by=[('Outer_acc_bal', 'mean'), ('Outer_acc_bal', 'max')], ascending=False)

Outer_acc                     Inner_acc                      \
                  mean       max       std      mean       max       std   
Classifier                                                                 
LinearSVC     0.948387  1.000000  0.050891  0.945185  0.959259  0.007570   
LDA           0.903226  0.967742  0.050435  0.898148  0.914815  0.010656   
AdaBoost      0.958065  1.000000  0.050549  0.951111  0.966667  0.009370   
RandomForest  0.941935  1.000000  0.045110  0.943333  0.955556  0.008198   
RBF           0.919355  0.967742  0.046249  0.916296  0.944444  0.015440   
KNN           0.896774  0.967742  0.064157  0.908889  0.929630  0.011342   
MLPC          0.893548  0.967742  0.056999  0.911852  0.940741  0.016172   
GaussianNB    0.558065  0.645161  0.048208  0.587407  0.644444  0.032156   
QDA           0.680645  0.774194  0.082523  0.735556  0.785185  0.022098   

             Outer_acc_bal                      Outer_f1                      \
                      mean       max       std      mean       max       std   
Classifier                                                                     
LinearSVC         0.944444  1.000000  0.069779  0.945898  1.000000  0.055029   
LDA               0.919571  0.969697  0.039946  0.906548  0.966139  0.047557   
AdaBoost          0.893813  1.000000  0.129515  0.951574  1.000000  0.056189   
RandomForest      0.883965  1.000000  0.120664  0.936614  1.000000  0.049686   
RBF               0.838258  0.958333  0.089633  0.911641  0.969892  0.048819   
KNN               0.815025  0.929293  0.099546  0.892450  0.953917  0.064382   
MLPC              0.789015  0.944444  0.117478  0.882501  0.965217  0.059490   
GaussianNB        0.613258  0.766414  0.114854  0.573790  0.659266  0.046672   
QDA               0.437753  0.551768  0.102823  0.629353  0.731336  0.086196   

             Outer_prec                     Outer_rec                      
                   mean       max       std      mean       max       std  
Classifier                                                                 
LinearSVC      0.951890  1.000000  0.048792  0.948387  1.000000  0.050891  
LDA            0.929680  0.970430  0.030113  0.903226  0.967742  0.050435  
AdaBoost       0.947672  1.000000  0.061626  0.958065  1.000000  0.050549  
RandomForest   0.942151  1.000000  0.050632  0.941935  1.000000  0.045110  
RBF            0.917895  0.983871  0.053645  0.919355  0.967742  0.046249  
KNN            0.905916  0.978495  0.060618  0.896774  0.967742  0.064157  
MLPC           0.885616  0.970430  0.067086  0.893548  0.967742  0.056999  
GaussianNB     0.650192  0.747670  0.052200  0.558065  0.645161  0.048208  
QDA            0.678147  0.774763  0.080583  0.680645  0.774194  0.082523

### With my runs LinearSVC performs best for both outer- and inner-loop accuracy scores, so we investigate a little further.

Look at the range of outer-loop results, and look at the HPs of the tuned models for each iteration

In [9]:
df.loc[df['Classifier']=='LinearSVC',:].dropna(axis=1, how='all').sort_values(by='Outer_acc', ascending=False)

,Classifier,Outer_acc,Inner_acc,Outer_acc_bal,Outer_f1,Outer_prec,Outer_rec,Unnamed: 0,tol,C,class_weight,max_iter,multi_class,penalty,dual
0,LinearSVC,1.000000,0.959259,1.000000,1.000000,1.000000,1.000000,3,0.018410,32.783846,NaN,500.0,ovr,l1,auto
0,LinearSVC,1.000000,0.940741,1.000000,1.000000,1.000000,1.000000,6,0.000974,91.483953,NaN,500.0,ovr,l1,auto
0,LinearSVC,1.000000,0.944444,1.000000,1.000000,1.000000,1.000000,9,0.021231,94.547738,balanced,500.0,ovr,l1,auto
0,LinearSVC,0.967742,0.940741,0.984848,0.967675,0.970430,0.967742,7,0.019465,53.828772,NaN,500.0,crammer_singer,l1,auto
0,LinearSVC,0.967742,0.940741,0.984848,0.968766,0.974194,0.967742,8,0.016741,91.897991,NaN,500.0,ovr,l1,auto
0,LinearSVC,0.935484,0.933333,0.929293,0.934178,0.945409,0.935484,0,0.006170,32.275699,NaN,500.0,ovr,l1,auto
0,LinearSVC,0.935484,0.948148,0.969697,0.936508,0.941935,0.935484,4,0.015042,38.966595,NaN,500.0,ovr,l1,auto
0,LinearSVC,0.935484,0.944444,0.929293,0.933983,0.944624,0.935484,5,0.069928,56.860969,balanced,500.0,ovr,l1,auto
0,LinearSVC,0.903226,0.944444,0.803030,0.890425,0.885484,0.903226,2,0.075975,69.596188,NaN,500.0,ovr,l1,auto
0,LinearSVC,0.838710,0.955556,0.843434,0.827445,0.856824,0.838710,1,0.012912,95.324891,NaN,500.0,crammer_singer,l1,auto


for my run it looks like `multi_class='ovr'`, `penalty='l1'` and `dual='auto'` worked well, as did a `tol < ~0.1`.

We can also look at mean values of C, and tol

In [10]:
df.loc[df['Classifier']=='LinearSVC', :].dropna(axis=1, how='all').sort_values(by='Outer_acc', ascending=False)[['C', 'tol']].mean()

C      65.756664
tol     0.025685
dtype: float64

### For reference, we look at the inner-loop results, to see if LinearSVC still had the highest average score while being tuned

In [11]:
df_il = pd.DataFrame()
for file in just_il:
    data = pd.read_csv(file_path+file)
    data['Classifier'] = file.split('_')[2]
    df_il = pd.concat([df_il, data])

cols_to_move = ['Classifier','value']
df_il = df_il[cols_to_move + [col for col in df_il.columns if col not in cols_to_move]]
# df_il.head()

In [12]:
df_il[cols_to_move].groupby('Classifier').agg(['max', 'mean', 'std']).sort_values(by=[('value', 'mean')],
                                                                           ascending=False)

value                    
                   max      mean       std
Classifier                                
LinearSVC     0.959259  0.892452  0.070789
AdaBoost      0.966667  0.890111  0.068264
RandomForest  0.955556  0.828407  0.145244
RBF           0.944444  0.789593  0.120477
KNN           0.929630  0.769207  0.128601
QDA           0.785185  0.670919  0.052368
GaussianNB    0.644444  0.587237  0.030723
MLPC          0.940741  0.521548  0.204278
LDA           0.914815  0.486548  0.363918

Yes, with the inner loop scores, LSVC still performs best on average. We can then use the successful parameters to narrow down our optuna parameter ranges to rerun these with a Leave-One-Polymer-Out (LOPO) cross validation to see how well LinearSVC can sort "new" plastics. This analysis is more generally called a Leave-One-Group-Out (LOGO) cross validation.

## Testing how well our model works with polymers it has never seen before

### $\color{red}{\text{Please run}}$ `5c-pirc_classification-LSVC_LOGO.py` $\color{red}{\text{before continuing!}}$

Here we will do another nested-cv, this time we first split our data 6:1, reserving one sample from each polymer species for our outer test.

Then we can use leave-one-group-out cv to remove a single polymer species from each of the inner training sets. The excluded species will then be used for the testing set. 

In [13]:
file_path = r'HP_Param_Scores/Optuna/Example/IndvModel/LSVC/'#/6-1_Split/'
filelist = [f for f in listdir(file_path) if isfile(join(file_path, f))]
logo_ol = [f for f in filelist if 'OuterLoop' in f]
logo_il = [f for f in filelist if 'OStudy' in f]

print(f'Files: {len(logo_ol)}')
print(f'{logo_ol[0]}')

Files: 7
20240627095128_OuterLoop_LinearSVC_6.csv


#### If we look at just the inner loop accuracy:

In [14]:
logo_df_il = pd.DataFrame()
for file in logo_il:
    data = pd.read_csv(file_path+file)
    data['Classifier'] = file.split('_')[2]
    logo_df_il = pd.concat([logo_df_il, data])

cols_to_move = ['Classifier','value']
logo_df_il = logo_df_il[cols_to_move + [col for col in logo_df_il.columns if col not in cols_to_move]]

logo_df_il[cols_to_move].groupby('Classifier').agg(['max', 'mean', 'std']).sort_values(by=[('value', 'max')],
                                                                           ascending=False)

value                    
                 max      mean       std
Classifier                              
LinearSVC   0.603488  0.549161  0.027121

I got a mean accuracy of 0.549161, which is not great! But...

#### If we look at the outer loop accuracy:

In [15]:
logo_df_ol = pd.DataFrame()
for file in logo_ol:
    data = pd.read_csv(file_path+file)
    data['Classifier'] = file.split('_')[2]
    logo_df_ol = pd.concat([logo_df_ol, data])

cols_to_move = ['Classifier','Outer_acc', 'Inner_acc', 'Outer_acc_bal',
                'Outer_f1', 'Outer_prec','Outer_rec']
logo_df_ol = logo_df_ol[cols_to_move+[col for col in logo_df_ol.columns if col not in cols_to_move]]

logo_df_ol.sort_values(by=['Inner_acc', 'Outer_acc'], ascending=False).head(10)

,Classifier,Outer_acc,Inner_acc,Outer_acc_bal,Outer_f1,Outer_prec,Outer_rec,Unnamed: 0,C,class_weight,max_iter,multi_class,penalty,dual,tol
0,LinearSVC,0.965116,0.603488,0.947222,0.966639,0.974630,0.965116,2,129.951047,NaN,1000,ovr,l1,auto,0.033673
0,LinearSVC,0.918605,0.603045,0.926389,0.919695,0.926396,0.918605,6,117.820821,NaN,1000,ovr,l1,auto,0.019788
0,LinearSVC,0.918605,0.602602,0.931250,0.918771,0.926971,0.918605,3,41.763873,balanced,1000,ovr,l1,auto,0.005503
0,LinearSVC,0.953488,0.600332,0.942014,0.953826,0.959197,0.953488,0,148.261244,NaN,1000,ovr,l1,auto,0.034912
0,LinearSVC,0.953488,0.596069,0.963542,0.954613,0.961914,0.953488,4,147.167688,NaN,1000,ovr,l1,auto,0.006645
0,LinearSVC,1.000000,0.592248,1.000000,1.000000,1.000000,1.000000,1,81.292379,NaN,1000,ovr,l1,auto,0.024111
0,LinearSVC,0.976744,0.568605,0.988889,0.976696,0.978112,0.976744,5,146.878469,NaN,1000,ovr,l1,auto,0.014375


In [16]:
logo_df_ol[cols_to_move].groupby('Classifier').agg(['max', 'mean', 'std']).sort_values(by=[('Outer_acc', 'max')],
                                                                           ascending=False)

Outer_acc                    Inner_acc                      \
                 max     mean       std       max      mean       std   
Classifier                                                              
LinearSVC        1.0  0.95515  0.029591  0.603488  0.595199  0.012438   

           Outer_acc_bal                    Outer_f1                      \
                     max      mean      std      max      mean       std   
Classifier                                                                 
LinearSVC            1.0  0.957044  0.02837      1.0  0.955748  0.029404   

           Outer_prec                     Outer_rec                     
                  max      mean       std       max     mean       std  
Classifier                                                              
LinearSVC         1.0  0.961031  0.026954       1.0  0.95515  0.029591

I see an outer accuracy of about 0.95515 isn't too shabby! Especially considering the LOGO training! From here we can take the mean or mode of each parameter from the above table to make our finalized model.